In [1]:
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sqlite3
import category_encoders as ce

import os
import sys

import numpy as np
import pandas as pd
import sqlite3
import category_encoders as ce
import re
import sklearn

from sklearn.feature_selection import chi2, SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer, CountVectorizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
polynomial_features= PolynomialFeatures(degree=2, interaction_only=True)

from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble 

from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


import warnings
warnings.filterwarnings('ignore')

from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [2]:
# src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
# sys.path.append(src_dir)

# from features.features import get_year, get_word_count, get_encoded_text
# from features.features import get_numeric_data ,get_combine_text ,get_reset_index,get_text_data 
# from features.features import SparseInteractions


In [4]:
#create the dataframe as df from the data source

# df = pd.read_csv()
# df = pd.read_excel()

conn = sqlite3.connect(r'C:\Users\MTGro\Desktop\coding\wineApp\db\wine_data.sqlite')
c = conn.cursor

df = pd.read_sql("select country  \
                 ,description     \
                 ,rating          \
                 ,price           \
                 ,province        \
                 ,title           \
                 ,winery from wine_data limit 20000", conn)
df.head(2)

,country,description,rating,price,province,title,winery
0,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Quinta dos Avidagos
1,Portugal,This wine is light in tannins and ripe in frui...,85,11.0,Douro,Quinta dos Avidagos 2015 Lote 138 Red (Douro),Quinta dos Avidagos


# Useful Functions for the Pipeline
The functions take a dataframe and return a dataframe so they can easily be used in the pipeline as needed. 

### Function: Reset Index of Dataframe
Useful if you need to loop starting at index 0 post train/test split.

In [5]:
def reset_index(dataframe):
    dataframe = dataframe.reset_index(inplace = False)
    return dataframe

#Create Transformer for pipeline
get_reset_index = FunctionTransformer(reset_index, validate=False)

### Function: Count the number of words in a column
Creates a new column named word_count

Specify the column for which you want a word count using the global variable word_count_column

In [6]:
text_column = 'description'
punc = [';'
        , "'"
        , '--'
        , ':'
        , '"'
        , "!"
        , "?"
        , '-'
        , ','
        , '.'
        , "("
        , ")"
        , '$'
        ,'`'
        ,'~'
       ]
        
        
        
def features(dataframe) :
    dataframe['word_count'] = dataframe[text_column].apply(lambda x : len(x.split()))
    dataframe['char_count'] = dataframe[text_column].apply(lambda x : len(x.replace(" ","")))
    dataframe['word_density'] = dataframe['word_count'] / (dataframe['char_count'] + 1)
    dataframe['punc_count'] = dataframe[text_column].apply(lambda x : len([a for a in x if a in punc]))
    dataframe['total_length'] = dataframe[text_column].apply(len)
    dataframe['capitals'] = dataframe[text_column].apply(lambda x: sum(1 for c in x if c.isupper()))
    dataframe['caps_vs_length'] = dataframe.apply(lambda row: float(row['capitals'])/float(row['total_length']),axis=1)
    dataframe['num_exclamation_marks'] =dataframe[text_column].apply(lambda x: x.count('!'))
    dataframe['num_question_marks'] = dataframe[text_column].apply(lambda x: x.count('?'))
    dataframe['num_punctuation'] = dataframe[text_column].apply(lambda x: sum(x.count(w) for w in '.,;:'))
    dataframe['num_symbols'] = dataframe[text_column].apply(lambda x: sum(x.count(w) for w in '*&$%'))
    dataframe['num_unique_words'] = dataframe[text_column].apply(lambda x: len(set(w for w in x.split())))
    dataframe['words_vs_unique'] = dataframe['num_unique_words'] / dataframe['word_count']
    dataframe["word_unique_percent"] =  dataframe["num_unique_words"]*100/dataframe['word_count']
    
    return dataframe

#Create Transformer for pipeline
get_word_count = FunctionTransformer(features, validate=False)

### Function: Combine Text Columns
Creates a new column named text

Specifiy the columns you want to combine in the global variable combine_text

In [7]:
combine_text = ['country','province','title','winery', 'description']

def combine_text_columns(dataframe):
    global combine_text
    text_data = dataframe[combine_text]
    
    # replace nans with blanks
    text_data.fillna("", inplace=True)
    
    # joins all of the text items in a row (axis=1)
    # with a space in between
    dataframe['text'] = text_data.apply(lambda x: " ".join(x), axis=1)
    
    return dataframe


#Create Transformer for pipeline using the combine_text_columns function
get_combine_text = FunctionTransformer(combine_text_columns, validate=False)

### Function: Category Encoder
pip install category_encoders / conda install -c conda-forge category_encoders

Specify the columns you want to encode in the variable category_columns

Specify the target column using the variable category_target if needed

Adjust the encoder as needed.

Takes in a dataframe and outputs a dataframe

In [8]:
category_columns = ['country','province','title','winery']
category_target = 'price'

# ce.BackwardDifferenceEncoder(cols=[...])
# ce.BaseNEncoder(cols=[...])
# ce.BinaryEncoder(cols=[...])
# ce.CatBoostEncoder(cols=[...])
# ce.HashingEncoder(cols=[...])
# ce.HelmertEncoder(cols=[...])
# ce.JamesSteinEncoder(cols=[...])
# ce.LeaveOneOutEncoder(cols=[...])
# ce.MEstimateEncoder(cols=[...])
# ce.OneHotEncoder(cols=[...])
# ce.OrdinalEncoder(cols=[...])
# ce.SumEncoder(cols=[...])
# ce.PolynomialEncoder(cols=[...])
# ce.TargetEncoder(cols=[...])
# ce.WOEEncoder(cols=[...])


def category_encode(dataframe):
    global category_columns
    global category_target
    x = dataframe[category_columns]
    y = dataframe[category_target]
    ce_ord = ce.JamesSteinEncoder(cols=category_columns)
    dataframe[category_columns] = ce_ord.fit_transform(x, y)
    return dataframe

#Create Transformer for pipeline
get_encoded_text = FunctionTransformer(category_encode, validate=False)

# Configure Text Vectorizer

### The following vectorizers can be configured: 

##### Count Vectorizer

##### TF-IDF vectorizer  

##### Hashing Vectorizer 

Modify the base_args and the specific args. 
The parameters are passed in to the vectorizer.

In [9]:
### sklearn.feature_extraction.text
cvec = CountVectorizer()
tfidf = TfidfVectorizer()
hvec = HashingVectorizer()

In [10]:
### Token Patterns
TOKENS_ALPHANUMERIC_d ='(?u)\\b\\w\\w+\\b' # Default
TOKENS_ALPHANUMERIC_1 = '[A-Za-z0-9]+(?=\\s+)'
TOKENS_ALPHANUMERIC_2 = r'\w{1,}'

### arguments applicable to all text vectorizers
base_args = {
        'encoding' : 'utf-8'
        ,'decode_error' : 'strict'
        ,'strip_accents' : None
        ,'lowercase' : True
        ,'preprocessor': None
        ,'tokenizer' : None
        ,'analyzer' : 'word'
        ,'stop_words': None
        ,'token_pattern' : TOKENS_ALPHANUMERIC_d
        ,'ngram_range' : (1,1)
    }

### specific to Count Vectorizer
cvec_args = {
        'max_df' : 1.0
        ,'min_df' : 1
        ,'max_features' : None
        ,'vocabulary': None
    }

### specific to Tfidf Vectorizer
tfidf_args = {
        'max_df' : 1.0
        ,'min_df' : 1
        ,'max_features' : None
        ,'vocabulary' : None
        ,'use_idf' : True
        ,'smooth_idf' : True
        ,'sublinear_tf' : False
    }

### specific to Hashing Vectorizer
hashing_args = {
        'n_features' : 1048576
        ,'norm' : 'l2'
        ,'alternate_sign' : True
    }

### Merge contents of dict2 and dict1 to dict3
cvec_params = {**base_args , **cvec_args}
tfidf_params = {**base_args , **tfidf_args}
hashing_params = {**base_args , **hashing_args}

In [11]:
#set the parameters for the text vectorizers
cvec.set_params(**cvec_params)
tfidf.set_params(**tfidf_params)
hvec.set_params(**hashing_params)
print('done')

done


# Feature Selection
Two select k best options:

chi2

f_classif

In [12]:
chi_k = 300
fs__kbest = SelectKBest(chi2, chi_k)
fs__f_classif = SelectKBest(f_classif, chi_k)

## Construct the text_features pipeline
Use variable text to declare text columns

Use comments to adjust vectorizer as needed

In [13]:
#declare which columns to will be unioned with the numeric features.
text = 'text'
get_text_data = FunctionTransformer(lambda x: x[text], validate=False)

In [14]:
#construct a pipeline 
text_features = Pipeline([
                    ('reset_index', get_reset_index),
                    ('combine_text',get_combine_text),
                    ('selector', get_text_data),
                    ('t_vectorizer', tfidf),
#                     ('c_vectorizer', cvec),
#                     ('h_vectorizer', hvec),
                    ('feature_selection', fs__kbest)
                ])

## Construct the numeric_features pipeline
Use variable numeric to list numeric columns

Use comments to adjust vectorizer as needed

In [16]:
#List the columns that will be encoded 
numeric= ['price', 'country','province','title','winery'
          ,'word_count'
          ,'char_count'
          ,'word_density'
          ,'punc_count'
          ,'total_length'
          ,'capitals'
          ,'caps_vs_length'
          ,'num_exclamation_marks'
          ,'num_question_marks'
          ,'num_punctuation'
          ,'num_symbols'
          ,'num_unique_words'
          ,'words_vs_unique'
          ,'word_unique_percent'
         ]

#transformer used to select the numeric columns to union with text features
get_numeric_data = FunctionTransformer(lambda x: x[numeric], validate=False)

In [17]:
#passes the dataframe through the set of transformers
#encodes the data
#selects the numeric columns

numeric_features = Pipeline([
                    ('reset_index', get_reset_index),
                    ('word_count', get_word_count),
                    ('encode_categories', get_encoded_text),
                    ('selector', get_numeric_data),
#                     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
#                     ('features', polynomial_features)
                ])

In [18]:
### Combine the text features with the numeric ceatures
union_features = Pipeline(steps =[('union', FeatureUnion(
                transformer_list = [
                    ('numeric_features', numeric_features),
                    ('text_features', text_features)
                ]))  
            ])

In [ ]:
classifiers = [
        LogisticRegression(C=1.0),
        MultinomialNB(),
        KNeighborsClassifier(20),
        #SVC(kernel="rbf", C=0.025, probability=True),
        #NuSVC(probability=True),
        DecisionTreeClassifier(),
        RandomForestClassifier(),
        AdaBoostClassifier(),
        GradientBoostingClassifier()
        ]

features = df.drop(['rating'], axis=1)

X = features
y = df['rating']
X_train, X_test, y_train, y_test = train_test_split(X, y
                                                   , test_size = .3
                                                   #, stratify=y
                                                   , random_state = 42
                                                   , shuffle=True
                                                   )    


for classifier in classifiers:
    pl = Pipeline(steps=[('preprocessor', union_features),
                      ('classifier', classifier)])
    
    pl.fit(X_train, y_train)   
    
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))




In [27]:
classifier = GradientBoostingClassifier()
param_grid = {
        'classifier__criterion' : ['friedman_mse'] #friedman_mse, mae, mse
        ,'classifier__n_estimators' : [3000]
        ,'classifier__max_depth' : [7]
        ,'classifier__learning_rate': [0.025,0.09]
        ,'classifier__random_state' : [42]
        ,'classifier__min_samples_leaf' : [21]
        ,'classifier__min_samples_split' : [27]
        ,'classifier__loss' : ['deviance']  #‘deviance’, ‘exponential’
#         ,'classifier__init' : [None]
        ,'classifier__max_features' : ['sqrt'] #None, 'auto', 'sqrt', 'log2'
#         ,'classifier__max_leaf_nodes': [None]
#         ,'classifier__min_impurity_decrease' : [0.0]
#         ,'classifier__min_impurity_split' : [None]
#         ,'classifier__min_weight_fraction_leaf' : [0.0]
#         ,'classifier__n_iter_no_change' : [None]
#         ,'classifier__subsample' : [1.0]
#         ,'classifier__tol' : [0.0001]
#         ,'classifier__validation_fraction' : [0.1]
#         ,'classifier__verbose' : [0]
#         ,'classifier__warm_start' : [False]
    }

In [28]:
pl = Pipeline(steps=[('preprocessor', union_features),
                  ('classifier', classifier)], verbose = True)

In [29]:
target = 'rating'
features = df.drop([target], axis=1)

X = features
y = df[target]


X_train, X_test, y_train, y_test = train_test_split(X, y
                                                   , test_size = .3
                                                   #, stratify = y
                                                   , random_state = 42
                                                   , shuffle=True
                                                   )


CV = GridSearchCV(pl, param_grid, n_jobs= -1)
                  
CV.fit(X_train, y_train)  
print(CV.best_params_)    
print(CV.best_score_)

# pl.fit(X_train, y_train)
# pl.score(X_train, y_train)

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   2.7s
[Pipeline] ........ (step 2 of 2) Processing classifier, total=14.3min
{'classifier__criterion': 'friedman_mse', 'classifier__learning_rate': 0.025, 'classifier__loss': 'deviance', 'classifier__max_depth': 7, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 21, 'classifier__min_samples_split': 27, 'classifier__n_estimators': 3000, 'classifier__random_state': 42}
0.19707142857142856


In [ ]:
#New Tests
# [Pipeline] ........ (step 2 of 2) Processing classifier, total= 1.3min
# {'classifier__learning_rate': 0.02, 'classifier__loss': 'huber', 'classifier__max_depth': 3, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 15, 'classifier__min_samples_split': 27, 'classifier__n_estimators': 3000}
# 0.6289488555219046

# [Pipeline] ........ (step 2 of 2) Processing classifier, total= 2.1min
# {'classifier__learning_rate': 0.02, 'classifier__loss': 'huber', 'classifier__max_depth': 7
# , 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 21, 'classifier__min_samples_split': 27
#             , 'classifier__n_estimators': 3000}
# 0.6299899436258168



#original Tests
##TEST 1
# [Pipeline] ........ (step 2 of 2) Processing classifier, total=  10.4s
# {'classifier__learning_rate': 0.01, 'classifier__max_depth': 5, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 7, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 500}
# 0.5591003524136465

##TEST 2
#[Pipeline] ........ (step 2 of 2) Processing classifier, total=   9.6s
# {'classifier__learning_rate': 0.01, 'classifier__max_depth': 5, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 13, 'classifier__min_samples_split': 13, 'classifier__n_estimators': 500}
# 0.5611780620229795

##TEST 3
# [Pipeline] ........ (step 2 of 2) Processing classifier, total=   9.5s
# {'classifier__learning_rate': 0.01, 'classifier__max_depth': 5, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 13, 'classifier__min_samples_split': 13, 'classifier__n_estimators': 500}
# 0.5615370173848946

# [Pipeline] ........ (step 2 of 2) Processing classifier, total=  14.4s
# {'classifier__learning_rate': 0.01, 'classifier__max_depth': 6, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 13, 'classifier__min_samples_split': 27, 'classifier__n_estimators': 600}
# 0.5625332421263227




In [ ]:
pl.fit(X_train, y_train)
pl.score(X_test, y_test)

In [ ]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [ ]:
print ("logloss: %0.3f " % multiclass_logloss(y_test, clf.predict_proba(X_test)))